In [1]:
# 1.a

sc = SparkContext.getOrCreate()
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
print type (wordsRDD)

<class 'pyspark.rdd.RDD'>


In [2]:
# 1.b

def makePlural(word):
    return (word + 's')
print makePlural('cat')

cats


In [3]:
# 1.c

pluralRDD = wordsRDD.map(makePlural)
print pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


In [4]:
# 1.d

pluralLambdaRDD = wordsRDD.map(lambda r: r + 's')
print pluralLambdaRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


In [5]:
# 1.e

pluralLengths = (pluralRDD.map(lambda a: len(a)).collect())
print pluralLengths

[4, 9, 4, 4, 4]


In [6]:
# 1.f

wordPairs = wordsRDD.map(lambda k: (k, 1))
print wordPairs.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


In [7]:
# 2.a

wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0} : {1}'.format(key, list(value))
    
sorted(wordsGrouped.mapValues(lambda v:list(v)).collect())

rat : [1, 1]
elephant : [1]
cat : [1, 1]


[('cat', [1, 1]), ('elephant', [1]), ('rat', [1, 1])]

In [8]:
# 2.b

wordCountsGrouped = wordsGrouped.map(lambda x: (x[0], sum(x[1])))
print wordCountsGrouped.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [9]:
# 2.c

wordCounts = wordPairs.reduceByKey(lambda a, b: (a+b))
print wordCounts.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [10]:
# 2.d

wordCountsCollected = wordsRDD.map(lambda p: (p, 1)).reduceByKey(lambda a, b: (a+b)).collect()
print wordCountsCollected
sorted(wordCountsCollected)

[('rat', 2), ('elephant', 1), ('cat', 2)]


[('cat', 2), ('elephant', 1), ('rat', 2)]

In [11]:
# 3.a

uniqueWords = len(wordCountsCollected)
print uniqueWords

3


In [12]:
# 3.b

from operator import add
totalCount = (wordCounts.map(lambda x: x[1]).reduce(add))
average = totalCount / float(uniqueWords)
print(totalCount)
print (round(average, 2))

5
1.67


In [14]:
# 4.a

def wordCount(wordListRDD):
    return wordListRDD.map(lambda a: (a, 1)).reduceByKey(lambda a, b: (a+b))
print wordCount(wordsRDD).collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


In [15]:
# 4.b

import re, string
def removePunctuation(text):
    output = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return output.lower().strip()
print removePunctuation('Hi, you!')
print removePunctuation('No under_score!')
print removePunctuation('*   Remove punctuation then spaces *')


hi you
no underscore
remove punctuation then spaces


In [16]:
# 4.c

fileName = "shakespeare.txt"

shakespeareRDD = (sc.textFile(fileName, 8).map(removePunctuation))
print '\n'.join(shakespeareRDD
          .zipWithIndex() # to (line, lineNum)
          .map(lambda (l, num): '{0}: {1}'.format(num, l)) # to 'lineNum: line'
          .take(15))

0: this is the 100th etext file presented by project gutenberg and
1: is presented in cooperation with world library inc from their
2: library of the future and shakespeare cdroms  project gutenberg
3: often releases etexts that are not placed in the public domain
4: 
5: shakespeare
6: 
7: this etext has certain copyright implications you should read
8: 
9: this electronic version of the complete works of william
10: shakespeare is copyright 19901993 by world library inc and is
11: provided by project gutenberg etext of illinois benedictine college
12: with permission  electronic and machine readable copies may be
13: distributed so long as such copies 1 are for your or others
14: personal use only and 2 are not distributed or used


In [17]:
# 4.d

shakespeareWordsRDD = shakespeareRDD.flatMap(lambda x: x.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()
print shakespeareWordsRDD.top(5)
print shakespeareWordCount

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']
946354


In [18]:
# 4.e

shakeWordsRDD = shakespeareWordsRDD.filter(lambda x: x!="")
shakeWordCount = shakeWordsRDD.count()
print shakeWordCount

901109


In [19]:
# 4.f

top15WordsAndCounts = wordCount(shakeWordsRDD).takeOrdered(15, key=lambda (a,b):-b)
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts))

the: 27645
and: 26733
i: 20683
to: 19198
of: 18180
a: 14613
you: 13650
my: 12480
that: 11122
in: 10967
is: 9598
not: 8725
for: 8245
with: 7996
me: 7768
